In [1]:
import numpy as np
import pandas as pd

import sympy
import sympy.physics.mechanics as me
from decimal import Decimal
from scipy.linalg import eigvals

In [2]:
# This is to calculate the Lengths of the cables for the Linearized Natural Frequency

# Create the variables
x, y, beta = me.dynamicsymbols('x, y, beta')

# Create the velocities
x_dot, y_dot, beta_dot = me.dynamicsymbols('x, y, beta', 1)

# Create the constants
m, k, L, g, H, c, D,t, Izz, k_beta, c_beta = sympy.symbols('m k L g H c D t Izz k_beta c_beta')
L_1_init, L_2_init, Le1, Le2 = sympy.symbols('L_1_init L_2_init Le1 Le2')

# Create the world frame
N = me.ReferenceFrame('N')

# Create the rod frame
B = N.orientnew('B', 'axis', [beta, N.z])

# Set the rotation of the rod frame
B.set_ang_vel(N, beta_dot * N.z)

# Create the Origin
O1 = me.Point('O_1')

# Set origin velocity to zero
O1.set_vel(N, 0 * N.x)

# Create the second attachment point
O2 = O1.locatenew('O_2', H * N.x)
O2.set_vel(N, 0 * N.x)

# Locate the point in the N frame
# P = me.Point('pen')
P = O1.locatenew('P', x * N.x + y * N.y)

# P.set_pos(O1, x * N.x + y * N.y)

# Set the point's velocity
P.set_vel(N, x_dot * N.x + y_dot * N.y)

# Create the rod center of mass
G = P.locatenew('G', D/2 * B.y)

# Set the velocity of G
G.v2pt_theory(P, N, B)

# Create the rod
I_rod = me.inertia(B, 0, 0, Izz)
rod = me.RigidBody('rod', G, B, m, (I_rod, G))

# Create the distance from the point to each attachment point
L1 = O1.pos_from(P).magnitude
L2 = O2.pos_from(P).magnitude
L1_vector = O1.pos_from(P).normalize
L2_vector = O2.pos_from(P).normalize

F1 = (Le1 - L1()) * k * L1_vector()
F2 = (Le2 - L2()) * k * L2_vector()

# Create the height from the center of gravity to the datum
h = G.pos_from(O1) & N.y

# The forces at the connection point
forceP = c * (x_dot + y_dot) * L1_vector() + c * (x_dot + y_dot) * L2_vector()

# The forces on the beta frame
forceB = -c_beta * beta_dot * N.z

rod.potential_energy = (-m * g * h + 0.5 * k * (L1() - L_1_init)**2 + 0.5 * k *(L2() - 
                        L_2_init)**2 + 0.5 * k_beta * beta**2)

Lag = me.Lagrangian(N, rod)

LM = me.LagrangesMethod(Lag, [x, y, beta], forcelist=[(P, forceP), (B, forceB)], frame=N)

# No damping
# LM = me.LagrangesMethod(Lag, [x, y, beta], frame=N)
EqMotion = LM.form_lagranges_equations()
lrhs = LM.rhs()

In [20]:
True_length(12,18)

(21.1617015459533, 19.0536003033547)

In [4]:
True_length(points[i][0],points[i][1])

NameError: name 'True_length' is not defined

In [5]:
points[1]

NameError: name 'points' is not defined

In [6]:
points_x = np.array([[6.5], [6.5], [6.5], [6.5], [6.5], [6.5], [6.5], [6.5],[7.5], [8.5],
                     [9.5],[10.5],[11.5],[12.5],[13.5],[14],[14],[14],[14],[14],[14],[14],
                     [14],[13.5],[12.5],[11.5],[10.5],[9.5],[8.5],[7.5]])

points_y = np.array([[5],[6],[7],[8],[9],[10],[11],[12],[12],[12],[12],[12],[12],[12],[12],
                     [12],[11],[10],[9],[8],[7],[6],[5],[4.5],[4.5],[4.5],[4.5],[4.5],[4.5],
                     [4.5]])
points = np.hstack([points_x, points_y])

In [16]:
def True_length(x,y):
    '''
    This calculates the True lengths of the cables, for any given X and Y the geometric cable
    lengths will not be correct because of the spring stretch. This calculates the cable length 
    that will stretch to the X and Y position. 
    
    Y has to be above 1 or the cables cannot reach that position. 
    
    The sympy solver is very slow it would be more efficent to use numpy,
    however this only has to be run once to get the lengths so the convience of this slow 
    algebric solver is ok.
    ''' 
    h = 20.0
    g = 9.81
    k = 10.0
    m = 1.0
    
    Fx = (-k*(-Le1 + sympy.sqrt(x**2 + y**2))*x/sympy.sqrt(x**2 + y**2) + 
         k*(h - x)*(-Le2 + np.sqrt((h - x)**2 + y**2))/sympy.sqrt((h - x)**2 + y**2))
    
    Fy = (-k*(-Le1 + sympy.sqrt(x**2 + y**2))*y/sympy.sqrt(x**2 + y**2) + 
          -k*(-Le2 + sympy.sqrt((h - x)**2 + y**2))*y/sympy.sqrt((h - x)**2 + y**2)) + m*g
    
    solution = sympy.solvers.solve([Fx,Fy])
    L1d = solution[Le1] 
    L2d = solution[Le2]
#     L1 = round(L1d,4)
#     L2 = round(L2d,4)
    return L1d, L2d

In [4]:
# Actual width
h = 20.0

# Actual Height
# Because of how this program gets the cable lengths 
# this will be +1 whatever height you put here.
U = 10.0

# How many divisions?
h_div = 10.0
U_div = 10.0
nx = int(h_div)
ny = int(U_div)
n = int(h_div * U_div)

# Gets the amount of grid squares
Area = h_div * U_div

# Breaks the actual dimensions of grid sizes
Squares_horz = h / h_div
Squares_vert = U / U_div

# Gets the locations of the centers of the grid squares
Centers_horz = Squares_horz / 2.0
Centers_vert = Squares_vert / 2.0

dimsx = np.array([(0)])
for i in range(nx):
    X = np.array([(Centers_horz + Squares_horz * i)]) 
    dimsx = np.vstack([dimsx,X])
dimsx_no_zero = dimsx[1:,:]
# dimsx_no_zero_final = np.vstack([dimsx_no_zero,dimsx_no_zero,dimsx_no_zero,dimsx_no_zero,dimsx_no_zero])

dimsy = np.array([(0)])
for i in range(ny):
    Y = np.array([(Centers_vert + Squares_vert * i + 1)]) 
    dimsy = np.vstack([dimsy,Y])
dimsy_no_zero = dimsy[1:,:]
# dimsx_no_zero_final = np.vstack([dimsx_no_zero,dimsx_no_zero,dimsx_no_zero,dimsx_no_zero,dimsx_no_zero])

dims = np.hstack([0,0])

for i in range(ny):
    a = np.zeros((nx, 1))
    a.fill(np.asscalar(dimsy_no_zero[i]))
    dims_temp = np.hstack([dimsx_no_zero, a])
    dims = np.vstack([dims,dims_temp])
dims_no_zero = dims[1:,:]

In [37]:
points_x = np.array([[6.5], [6.5], [6.5], [6.5], [6.5], [6.5], [6.5], [6.5],[7.5], [8.5],
                     [9.5],[10.5],[11.5],[12.5],[13.5],[14],[14],[14],[14],[14],[14],[14],
                     [14],[13.5],[12.5],[11.5],[10.5],[9.5],[8.5],[7.5]])

points_y = np.array([[5],[6],[7],[8],[9],[10],[11],[12],[12],[12],[12],[12],[12],[12],[12],
                     [12],[11],[10],[9],[8],[7],[6],[5],[4.5],[4.5],[4.5],[4.5],[4.5],[4.5],
                     [4.5]])
points = np.hstack([points_x, points_y])
# np.shape(points_y)
n = np.shape(points)[0]
points[0][0]

6.5

In [5]:
True_length(10,7.5)

(11.6825, 11.6825)

In [6]:
###### ORIGINAL ########

# Lengths = np.array([(0,0)])

# for i in range(n):
#         X,Y = dims_no_zero[i,:]
#         H = 20
#         L1 = np.sqrt(Y**2 + X**2)
#         L2 = np.sqrt(Y**2 + (H - X)**2)
#         Lengths = np.append(Lengths, [(L1, L2)], axis=0)
# Lengths_no_zero = Lengths[1:,:]

In [6]:
Lengths = np.array([(0,0)])

for i in range(n):
        X,Y = dims_no_zero[i,:]
        L1, L2 = True_length(X,Y)
        print(i)
        Lengths = np.append(Lengths, [(L1, L2)], axis=0)
Lengths_no_zero = Lengths[1:,:]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [7]:
print(dims_no_zero[25], Lengths_no_zero[25])

[ 11.    3.5] [ 10.0874   8.168 ]


In [8]:
nat_freq_high_eigen = np.array([(0)])
nat_freq_medium_eigen = np.array([(0)])
nat_freq_low_eigen = np.array([(0)])
damp_low = np.array([(0)])
damp_medium = np.array([(0)])
damp_high = np.array([(0)])

for i in range(n):
    
    operating_point = {x:dims_no_zero[i,0], y:dims_no_zero[i,1], beta:0, x_dot:0, y_dot:0, beta_dot:0}

    A, B, inp_vec = LM.linearize([x,y,beta], [x_dot, y_dot, beta_dot],
                                 op_point=operating_point,
                                 A_and_B=True)
    A_Sub = A.subs({k:10.0,m:1.0,g:9.81, L_1_init:Lengths_no_zero[i,0],L_2_init:Lengths_no_zero[i,1],
                    H:20.0, c:1.0,c_beta:1.0,x_dot:0.0,y_dot:0.0, beta_dot:0.0, D:1.0,Izz:0.33,k_beta:1.0})
    test_matrix = A_Sub.doit()
    test_object = np.array(test_matrix)
    test = test_object.astype(float)
    eigenvals, eigenvects = np.linalg.eig(test)
    eigen = np.unique(np.abs(eigenvals)*(1.0/(2*np.pi)))
    damping1 = -np.real(eigen[0]) / eigen[0]
    damping2 = -np.real(eigen[1]) / eigen[1]
    damping3 = -np.real(eigen[2]) / eigen[2]
    damping = np.array([(damping1,damping2,damping3)])
    damping_index = np.argsort(damping)
    eigen_index = np.argsort(eigen)
    highd, mediumd, lowd = damping[damping_index][::-1][:3][0:3]
    high, medium, low = eigen[eigen_index][::-1][:3][0:3]
    
    print(i)
    
    nat_freq_high_eigen = np.append(nat_freq_high_eigen, [(high)], axis=0)
    nat_freq_medium_eigen = np.append(nat_freq_medium_eigen, [(medium)], axis=0)
    nat_freq_low_eigen = np.append(nat_freq_low_eigen, [(low)], axis=0)
    damp_high = np.append(damp_high, [(highd)], axis=0)
    damp_medium = np.append(damp_medium, [(mediumd)], axis=0)
    damp_low = np.append(damp_low, [(lowd)], axis=0)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
nat_freq_high_eigen_no_zero = nat_freq_high_eigen[1:]
nat_freq_medium_eigen_no_zero = nat_freq_medium_eigen[1:]
nat_freq_low_eigen_no_zero = nat_freq_low_eigen[1:]

In [26]:
# np.savetxt("nat_freq_high_eigen_no_zero.csv", nat_freq_high_eigen_no_zero, delimiter=",")
# np.savetxt("nat_freq_medium_eigen_no_zero.csv", nat_freq_medium_eigen_no_zero, delimiter=",")
# np.savetxt("nat_freq_low_eigen_no_zero.csv", nat_freq_low_eigen_no_zero, delimiter=",")

In [ ]:
def eq_of_motion(w, t, p):
    """
    Defines the differential equations for the coupled spring-mass system.

    Arguments:
        w :  vector of the state variables:
                  w = [x, theta, x_dot, theta_dot]
        t :  time
        p :  vector of the parameters:
                  p = [m1, m2, k, l, g, wf]
    
    Returns:
        sysODE : An list representing the system of equations of motion as 1st order ODEs
    """
    x, y, beta, x_dot, y_dot, beta_dot = w
    m, k, g, H, c, D,t, Izz, k_beta, c_beta, L_1_init, L_2_init = p

    # Create sysODE = (x', theta', x_dot', theta_dot'):
    sysODE = [x_dot,
              y_dot,
              beta_dot,
              (-D*m*sin(beta)*beta_dot**2/2 + D*m*(-D*g*m*sin(beta)/2 + D*(-D*m*sin(beta)*beta_dot**2/2 + c*(H - x)*(x_dot + y_dot)/sqrt((H - x)**2 + y**2) - c*(x_dot + y_dot)*x/sqrt(x**2 + y**2) - 1.0*k*(-H + x)*(-L_2_init + sqrt((H - x)**2 + y**2))/sqrt((H - x)**2 + y**2) - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*x/sqrt(x**2 + y**2))*cos(beta)/2 + D*(D*m*cos(beta)*beta_dot**2/2 - c*(x_dot + y_dot)*y/sqrt(x**2 + y**2) - c*(x_dot + y_dot)*y/sqrt((H - x)**2 + y**2) + g*m - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*y/sqrt(x**2 + y**2) - 1.0*k*(-L_2_init + sqrt((H - x)**2 + y**2))*y/sqrt((H - x)**2 + y**2))*sin(beta)/2 - c_beta*beta_dot - 1.0*k_beta*beta - m*(-D*(-sin(beta)*beta_dot*x_dot + cos(beta)*beta_dot*y_dot)/2 + D*sin(beta)*beta_dot*x_dot/2 - D*cos(beta)*beta_dot*y_dot/2)/2 + m*(-D*(-sin(beta)*x_dot + cos(beta)*y_dot)*beta_dot/2 + D*sin(beta)*beta_dot*x_dot/2 - D*cos(beta)*beta_dot*y_dot/2)/2)*cos(beta)/(2*(-D**2*m*sin(beta)**2/4 - D**2*m*cos(beta)**2/4 + D**2*m/4 + Izz)) + c*(H - x)*(x_dot + y_dot)/sqrt((H - x)**2 + y**2) - c*(x_dot + y_dot)*x/sqrt(x**2 + y**2) - 1.0*k*(-H + x)*(-L_2_init + sqrt((H - x)**2 + y**2))/sqrt((H - x)**2 + y**2) - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*x/sqrt(x**2 + y**2))/m,
              (D*m*cos(beta)*beta_dot**2/2 + D*m*(-D*g*m*sin(beta)/2 + D*(-D*m*sin(beta)*beta_dot**2/2 + c*(H - x)*(x_dot + y_dot)/sqrt((H - x)**2 + y**2) - c*(x_dot + y_dot)*x/sqrt(x**2 + y**2) - 1.0*k*(-H + x)*(-L_2_init + sqrt((H - x)**2 + y**2))/sqrt((H - x)**2 + y**2) - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*x/sqrt(x**2 + y**2))*cos(beta)/2 + D*(D*m*cos(beta)*beta_dot**2/2 - c*(x_dot + y_dot)*y/sqrt(x**2 + y**2) - c*(x_dot + y_dot)*y/sqrt((H - x)**2 + y**2) + g*m - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*y/sqrt(x**2 + y**2) - 1.0*k*(-L_2_init + sqrt((H - x)**2 + y**2))*y/sqrt((H - x)**2 + y**2))*sin(beta)/2 - c_beta*beta_dot - 1.0*k_beta*beta - m*(-D*(-sin(beta)*beta_dot*x_dot + cos(beta)*beta_dot*y_dot)/2 + D*sin(beta)*beta_dot*x_dot/2 - D*cos(beta)*beta_dot*y_dot/2)/2 + m*(-D*(-sin(beta)*x_dot + cos(beta)*y_dot)*beta_dot/2 + D*sin(beta)*beta_dot*x_dot/2 - D*cos(beta)*beta_dot*y_dot/2)/2)*sin(beta)/(2*(-D**2*m*sin(beta)**2/4 - D**2*m*cos(beta)**2/4 + D**2*m/4 + Izz)) - c*(x_dot + y_dot)*y/sqrt(x**2 + y**2) - c*(x_dot + y_dot)*y/sqrt((H - x)**2 + y**2) + g*m - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*y/sqrt(x**2 + y**2) - 1.0*k*(-L_2_init + sqrt((H - x)**2 + y**2))*y/sqrt((H - x)**2 + y**2))/m,
              (-D*g*m*sin(beta)/2 + D*(-D*m*sin(beta)*beta_dot**2/2 + c*(H - x)*(x_dot + y_dot)/sqrt((H - x)**2 + y**2) - c*(x_dot + y_dot)*x/sqrt(x**2 + y**2) - 1.0*k*(-H + x)*(-L_2_init + sqrt((H - x)**2 + y**2))/sqrt((H - x)**2 + y**2) - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*x/sqrt(x**2 + y**2))*cos(beta)/2 + D*(D*m*cos(beta)*beta_dot**2/2 - c*(x_dot + y_dot)*y/sqrt(x**2 + y**2) - c*(x_dot + y_dot)*y/sqrt((H - x)**2 + y**2) + g*m - 1.0*k*(-L_1_init + sqrt(x**2 + y**2))*y/sqrt(x**2 + y**2) - 1.0*k*(-L_2_init + sqrt((H - x)**2 + y**2))*y/sqrt((H - x)**2 + y**2))*sin(beta)/2 - c_beta*beta_dot - 1.0*k_beta*beta - m*(-D*(-sin(beta)*beta_dot*x_dot + cos(beta)*beta_dot*y_dot)/2 + D*sin(beta)*beta_dot*x_dot/2 - D*cos(beta)*beta_dot*y_dot/2)/2 + m*(-D*(-sin(beta)*x_dot + cos(beta)*y_dot)*beta_dot/2 + D*sin(beta)*beta_dot*x_dot/2 - D*cos(beta)*beta_dot*y_dot/2)/2)/(-D**2*m*sin(beta)**2/4 - D**2*m*cos(beta)**2/4 + D**2*m/4 + Izz)
             ]
    return sysODE

In [ ]:
from scipy.integrate import odeint
import numpy as np
from numpy import sin, cos, sqrt
import seaborn as sns
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# nat_freq = np.array([(0)])
nat_freq_high = np.array([(0)])
nat_freq_medium = np.array([(0)])
nat_freq_low = np.array([(0)])

damping = np.array([(0)])

for i in range(n):
    # Set up simulation parameters 
    m = 1.0
    k = 10.0 
    # L = 1.0 
    g = 9.81 
    H = 20.0 
    c = 1.0 
    D = 1.0 
    Izz = 0.3333
    k_beta = 1.0
    c_beta = 1.0
    # L_1_init = 5.38343814
    # L_2_init = 16.70273649
    L_1_init = Lengths_no_zero[i,0]
    L_2_init = Lengths_no_zero[i,1]

    # ODE solver parameters
    abserr = 1.0e-9
    relerr = 1.0e-9
    max_step = 0.01
    stoptime = 50.0
    numpoints = 10000

    # Create the time samples for the output of the ODE solver
    t = np.linspace(0.0, stoptime, numpoints)

    # Initial conditions
    x_init = dims_no_zero[i,0] + 0.5                        # initial position
    # x_init = -6.25
    x_dot_init = 0.0                    # initial velocity
    y_init = dims_no_zero[i,1] + 0.5                  # initial angle
    # y_init = 3.8625
    y_dot_init = 0.0                # initial angular velocity
    beta_init = 0.0                        # initial position
    beta_dot_init = 0.0 

    # wf = np.np.sqrt(k / m1)                # forcing function frequency
    # Pack the parameters and initial conditions into arrays 
    p = [m, k, g, H, c, D, t, Izz, k_beta, c_beta, L_1_init, L_2_init]
    x0 = [x_init, y_init, beta_init, x_dot_init, y_dot_init, beta_dot_init]
    resp = odeint(eq_of_motion, x0, t, args=(p,), atol=abserr, rtol=relerr,  hmax=max_step)
    freq, mag = CRAWLAB_fft(resp[:,2],t,False)
#     freq_index = np.argmax(mag)
    # Should return the zero crossings
    zeros = get_zero_crossings(t, np.degrees(resp[:,2]))

    # Get the peaks of the response
    localMaxes, localMax_Times, localMins, localMin_Times = get_local_Extrema(t, np.degrees(resp[:,2]))

    # Calculate the damping ratio from both the local maxes and the local minimums
    zeta_maxes = log_dec(localMaxes[0],localMaxes[-1],len(localMaxes)-1)
    zeta_mins = log_dec(localMins[0],localMins[-1],len(localMins)-1)

    # Average the damping ratio from the positive and negative sides of the plot.
    zeta_fromLogDec = np.mean([zeta_maxes, zeta_mins]) 

    damping = np.append(damping, [(zeta_fromLogDec)], axis=0)
    
#     nat_freq = np.append(nat_freq, [(freq[freq_index])], axis=0)
    localMaxes, localMax_Times, localMins, localMin_Times = get_local_Extrema(freq,mag)
    
    first_max = np.argmax(localMaxes)
    localMaxes[first_max] = 0
    high_mode = localMax_Times[first_max]
    nat_freq_high = np.append(nat_freq_high, [(high_mode)], axis=0)

    second_max = np.argmax(localMaxes)
    localMaxes[second_max] = 0
    medium_mode = localMax_Times[second_max]
    nat_freq_medium = np.append(nat_freq_medium, [(medium_mode)], axis=0)

    third_max = np.argmax(localMaxes)
    low_mode = localMax_Times[third_max]
    nat_freq_low = np.append(nat_freq_low, [(low_mode)], axis=0)
    print(i)
#     print('High Mode:{}'.format(high_mode))
#     print('Medium Mode:{}'.format(medium_mode))
#     print('Low Mode:{}'.format(low_mode))

In [ ]:
nat_freq_high_no_zero = nat_freq_high[1:]
nat_freq_medium_no_zero = nat_freq_medium[1:]
nat_freq_low_no_zero = nat_freq_low[1:]
damping_no_zero = damping[1:]
# nat_freq_no_zero
# damping_no_zero

In [ ]:
np.savetxt("nat_freq_high.csv", nat_freq_high_no_zero, delimiter=",")
np.savetxt("nat_freq_medium.csv", nat_freq_medium_no_zero, delimiter=",")
np.savetxt("nat_freq_low.csv", nat_freq_low_no_zero, delimiter=",")

In [ ]:
sns.set(context="paper", font="monospace")
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
plt.figure(0)
Heat1 = np.hstack((np.fliplr(dims_no_zero), np.reshape(nat_freq_high_no_zero,(n,1))))
df = pd.DataFrame(Heat1, columns=list('YXF'))
Heats1 = df.pivot("Y", "X", "F")
# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heats1, annot=False, fmt="d", linewidths=0, cmap="inferno", xticklabels=False, yticklabels=False,
           vmin=0, vmax=1.5)
plt.title('Low Mode Natural Frequency Across Workspace', y = 1.05)
plt.savefig("Low_Mode_Natural_Frequency.pdf")
print("The average is: {}".format(np.average(nat_freq_high_no_zero)))

In [ ]:
nat_freq_high_no_zero
nat_freq_low_no_zero[10]

In [ ]:
fig1 = plt.figure(1)
ax1 = fig1.gca(projection='3d')         # set the 3d axes


ax1.plot_trisurf(dims_no_zero[:,0], dims_no_zero[:,1], nat_freq_high_no_zero, cmap="YlGnBu", 
                linewidth=0.0)
ax1.set_axis_bgcolor('white')
plt.title('Low Mode Natural Frequency Across Workspace')
# plt.savefig("Low_Mode_Natural_Frequency.pdf")

In [ ]:
plt.figure(2)
Heat2 = np.hstack((np.fliplr(dims_no_zero), np.reshape(nat_freq_medium_no_zero,(n,1))))
df = pd.DataFrame(Heat2, columns=list('YXF'))

Heats2 = df.pivot("Y", "X", "F")

# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heats2, annot=False, fmt="d", linewidths=0, cmap="inferno", xticklabels=False, yticklabels=False,
           vmin=0, vmax=1.5)
plt.title('Medium Mode Natural Frequency Across Workspace', y = 1.05)
plt.savefig("Medium_Mode_Natural_Frequency.pdf")
print("The average is: {}".format(np.average(nat_freq_medium_no_zero)))

In [ ]:
fig3 = plt.figure(3)
ax3 = fig3.gca(projection='3d')

ax3.set_axis_bgcolor('white')
ax3.plot_trisurf(dims_no_zero[:,0], dims_no_zero[:,1], nat_freq_medium_no_zero, cmap="YlGnBu", linewidth=0.0)
plt.title('Medium Mode Natural Frequency Across Workspace')
# plt.savefig("Medium_Mode_Natural_Frequency.pdf")

In [ ]:
plt.figure(4)
Heat3 = np.hstack((np.fliplr(dims_no_zero), np.reshape(nat_freq_low_no_zero,(n,1))))
df = pd.DataFrame(Heat3, columns=list('YXF'))

Heats3 = df.pivot("Y", "X", "F")

# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heats3, annot=False, fmt="d", linewidths=0, cmap="inferno", xticklabels=False, yticklabels=False,
           vmin=0, vmax=1.5)
plt.title('High Mode Natural Frequency Across Workspace', y = 1.05)
plt.savefig("High_Mode_Natural_Frequency.pdf")
print("The average is: {}".format(np.average(nat_freq_low_no_zero)))

In [ ]:
fig5 = plt.figure(5)
ax5 = fig5.gca(projection='3d')

ax5.set_axis_bgcolor('white')
ax5.plot_trisurf(dims_no_zero[:,0], dims_no_zero[:,1], nat_freq_low_no_zero, 
                cmap="YlGnBu", linewidth=0.0)

In [ ]:
plt.figure(6)
Heatd = np.hstack((np.fliplr(dims_no_zero), np.reshape(damping_no_zero,(n,1))))
dfd = pd.DataFrame(Heatd, columns=list('YXD'))

Heatsd = dfd.pivot("Y", "X", "D")

# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heatsd, annot=False, fmt="d", linewidths=0, cmap="YlGnBu", xticklabels=False, yticklabels=False)
plt.title('Damping Ratio Across Workspace', y = 1.05)
plt.savefig("Damping_Ratio.pdf")
print("The average is: {}".format(np.nanmean(damping_no_zero)))

In [ ]:
fig7 = plt.figure(7)
ax7 = fig7.gca(projection='3d')

ax7.set_axis_bgcolor('white')
ax7.plot_trisurf(dims_no_zero[:,0], dims_no_zero[:,1], damping_no_zero, 
                cmap="YlGnBu", linewidth=0.1)

In [ ]:
plt.figure(0)
Heat4 = np.hstack((np.fliplr(dims_no_zero), np.reshape(nat_freq_high_eigen_no_zero,(n,1))))
df = pd.DataFrame(Heat4, columns=list('YXF'))
Heats4 = df.pivot("Y", "X", "F")
# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heats4, annot=False, fmt="d", linewidths=0, cmap="inferno", xticklabels=False, yticklabels=False,
           )
plt.title('Low Mode Natural Frequency Across Workspace Eigenvalue', y = 1.05)
# plt.savefig("Low_Mode_Natural_Frequency_eigen.pdf")
print("The average is: {}".format(np.average(nat_freq_high_eigen_no_zero)))

In [ ]:
plt.figure(0)
Heat5 = np.hstack((np.fliplr(dims_no_zero), np.reshape(nat_freq_medium_eigen_no_zero,(n,1))))
df = pd.DataFrame(Heat5, columns=list('YXF'))
Heats5 = df.pivot("Y", "X", "F")
# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heats5, annot=False, fmt="d", linewidths=0, cmap="inferno", xticklabels=False, yticklabels=False)
plt.title('Medium Mode Natural Frequency Across Workspace Eigenvalue', y = 1.05)
# plt.savefig("Medium_Mode_Natural_Frequency_eigen.pdf")
print("The average is: {}".format(np.average(nat_freq_medium_eigen_no_zero)))

In [ ]:
plt.figure(0)
Heat6 = np.hstack((np.fliplr(dims_no_zero), np.reshape(nat_freq_low_eigen_no_zero,(n,1))))
df = pd.DataFrame(Heat6, columns=list('YXF'))
Heats6 = df.pivot("Y", "X", "F")
# Draw a heatmap with the numeric values in each cell
sns.heatmap(Heats6, annot=False, fmt="d", linewidths=0, cmap="inferno", xticklabels=False, yticklabels=False)
plt.title('High Mode Natural Frequency Across Workspace Eigenvalue', y = 1.05)
# plt.savefig("High_Mode_Natural_Frequency_eigen.pdf")
print("The average is: {}".format(np.average(nat_freq_low_eigen_no_zero)))

In [ ]:
def CRAWLAB_fft(data,time,plotflag):
    ''' Function to get the FFT for a response
    #
    # Inputs:
    #   time = time array corresponding to the data
    #   data = the response data array (only pass a single dimension/state at at time)
    #   plotflag = will plot the FFT if nonzero
    #   
    # Output:
    #   fft_freq = an array of the freqs used in the FFT
    #   fft_mag = an array of the amplitude of the FFT at each freq in fft_freq
    #
    # Created: 03/28/14
    #   - Joshua Vaughan
    #   - joshua.vaughan@louisiana.edu
    #   - http://www.ucs.louisiana.edu/~jev9637
    ######################################################################################
    '''
    
    from scipy.fftpack import fft
    
    # correct for any DC offset
    offset = np.mean(data) 

    # Get the natural frequency
    sample_time = time[1] - time[0]
    n = len(data)

    fft_mag = fft((data - offset)*np.hanning(len(data)))
    fft_freq = np.linspace(0.0, 1.0/(2.0*sample_time), n/2)
    
    # Only return the "useful" part of the fft
    fft_mag = 2.0/n * np.abs(fft_mag[0:n/2])
    
    if plotflag:
        # Plot the relationshiop
        #   Many of these setting could also be made default by the .matplotlibrc file
        fig = plt.figure(figsize=(6,4))
        ax = plt.gca()
        plt.subplots_adjust(bottom=0.17,left=0.17,top=0.96,right=0.96)
        plt.setp(ax.get_ymajorticklabels(),fontsize=18)
        plt.setp(ax.get_xmajorticklabels(),fontsize=18)
        ax.spines['right'].set_color('none')
        ax.spines['top'].set_color('none')
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')
        ax.grid(True,linestyle=':',color='0.75')
        ax.set_axisbelow(True)

        plt.xlabel('Frequency (Hz)',fontsize=22,labelpad=8)
        plt.ylabel('FFT magnitude',fontsize=22,labelpad=10)
    
        plt.plot(fft_freq, fft_mag, linewidth=2, linestyle='-')
        
        # Adjust the page layout filling the page using the new tight_layout command
        plt.tight_layout(pad=0.5)
        plt.show()
    
    return fft_freq, fft_mag

In [ ]:
# First define all the functions we'll need.
def log_dec(peak1,peak2,num_cycles):
    '''##########################################################################################
    # log_dec.py
    #
    # Script to compute damping ratio using log dec method
    #
    # Inputs:
    #   peak1 = the amplitude of the first peak
    #   peak2 = the amplitude of the Nth peak
    #   num_cycles = the number of periods between two peaks
    # 
    # Output:
    #   zeta = the damping ratio
    #
    # NOTE: Plotting is set up for output, not viewing on screen.
    #       So, it will likely be ugly on screen. The saved PDFs should look
    #       better.
    #
    # Created: 03/28/14
    #   - Joshua Vaughan
    #   - joshua.vaughan@louisiana.edu
    #   - http://www.ucs.louisiana.edu/~jev9637
    #
    # Modified:
    #   *
    #
    ######################################################################################
    '''
    import numpy as np

    delta = 1./num_cycles*np.log(peak1 / peak2)

    zeta = 1./np.sqrt(1 + (2 * np.pi/delta)**2)
    
    return zeta
    

def get_local_Extrema(time,data):
    ''' # Function to get the local extrema for a response
    #
    # Inputs:
    #   time = time array corresponding to the data
    #   data = the response data array (only pass a single dimension/state at at time)
    #
    # Output:
    #   localMaxes = the amplitude of the local maxes
    #   localMax_Times = the times of the local maxes
    #
    # Created: 03/28/14
    #   - Joshua Vaughan
    #   - joshua.vaughan@louisiana.edu
    #   - http://www.ucs.louisiana.edu/~jev9637
    ######################################################################################
    '''
    from scipy import signal
    
    # Get local maximums
    localMax_indexes = signal.argrelextrema(data, np.greater)
    localMaxes = data[localMax_indexes]
    localMax_Times = time[localMax_indexes]

    # Get local minimums
    localMin_indexes = signal.argrelextrema(data, np.less)
    localMins = data[localMin_indexes]
    localMin_Times = time[localMin_indexes]
    
    return localMaxes, localMax_Times, localMins, localMin_Times
    

def get_zero_crossings(time,data):
    ''' Function to get the local extrema for a response
    #
    # Inputs:
    #   time = time array corresponding to the data
    #   data = the response data array (only pass a single dimension/state at at time)
    #   
    # Output:
    #   zeros = an array of the times of the zero crossings
    #
    # Created: 03/28/14
    #   - Joshua Vaughan
    #   - joshua.vaughan@louisiana.edu
    #   - http://www.ucs.louisiana.edu/~jev9637
    ######################################################################################
    '''
    
    # create an empty zeros array
    zeros = []
    
    for index in range(len(time)-1):
        if np.sign(data[index]) != np.sign(data[index + 1]):
            zeros.append(time[index])
    
    return zeros

In [ ]:
freq, mag = CRAWLAB_fft(resp[:,2],t,False)

In [ ]:
plt.figure(6)
plt.plot(freq, mag)
plt.title('FFT')
plt.xlabel('frequency')
plt.ylabel('Magnitude')
plt.xlim(0, 5)
freq_index = np.argmax(mag)
print freq[freq_index]

In [ ]:
localMaxes, localMax_Times, localMins, localMin_Times = get_local_Extrema(freq,mag)

In [ ]:
first_max = np.argmax(localMaxes)
localMaxes[first_max] = 0
high_mode = localMax_Times[first_max]
nat_freq_high = np.append(nat_freq_high, [(high_mode)], axis=0)

second_max = np.argmax(localMaxes)
localMaxes[second_max] = 0
medium_mode = localMax_Times[second_max]
nat_freq_medium = np.append(nat_freq_medium, [(medium_mode)], axis=0)

third_max = np.argmax(localMaxes)
low_mode = localMax_Times[third_max]
nat_freq_low = np.append(nat_freq_low, [(low_mode)], axis=0)

# print(high_mode, medium_mode, low_mode)

In [ ]:
FFT_maxes = np.hstack((np.reshape(localMaxes, (12,1)), np.reshape(localMax_Times, (12,1))))